In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
from scipy.spatial import ConvexHull

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir3='/Volumes/My Passport/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
ds=xr.open_dataset(dir2+'merra2.ar.labels.model.1980.nc')
labels=ds['ar_labeled']

ds=xr.open_dataset(dir2+'era_5_gridareas.nc')
areas=ds['cell_area']/1e6

ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas1=ds['cell_area']/1e6

yrs=np.arange(1980,2020,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year) 
    
    year_length=365
    if year%4==0:
        year_length=366
        
    dates=[dt.datetime(year,1,1,0)+dt.timedelta(hours=6*x) for x in range(4*year_length)]

    if 1==1:
        ds=xr.open_dataset('/Users/ahenny/Downloads/ERA5.ar_tag.Reid500.1hr.'+str(year)+'0101-'+str(year)+'1231.nc')
        
        #ds1=xr.open_dataset('/Users/ahenny/Downloads/ERA5.ar_tag.Mundhenk_v3.1hr.19800101-19801231.nc')
        #ds=xr.open_dataset('/Users/ahenny/Downloads/ERA5.ar_tag.TempestLR.1hr.'+str(year)+'.nc')
        #ds['lat']=ds1.lat.values
        #ds['lon']=ds1.lon.values
        
        ar=ds['ar_binary_tag'].sel(time=dates)
        #lons_east=[x for x in ar.lon.values if x>=180.]
        #lons_west=[x for x in ar.lon.values if x<180.]
        
        #test=areas.where(ar[0,:,:]==1).sum().values.tolist()
        
        for j in range(len(dates)):
            #print(j)
            ar_sel=ar.sel(time=dates[j])
            #First - convert to [-180,180)
        
            #ar_west=ar_sel.sel(lon=lons_west)
            #ar_east=ar_sel.sel(lon=lons_east)
            #ar_east['lon']=[x-360. for x in lons_east]
            #ar_sel=xr.concat([ar_east,ar_west],dim='lon')

            ar_sel=ar_sel.interp(lat=labels.lat.values,lon=labels.lon.values,method='nearest')
        
            #test1=areas1.where(ar_sel>=1).sum().values.tolist()
            #print(test1/test)
            #sys.exit()
            
            if j==0:
                ar_concat=ar_sel
            else:
                ar_concat=xr.concat([ar_concat,ar_sel],dim='time')
        
        dk=xr.Dataset()
        dk['ar']=(('time','lat','lon'),ar_concat.values)
        
        dk.coords['time']=ar_concat.time.values
        dk.coords['lat']=ar_concat.lat.values
        dk.coords['lon']=ar_concat.lon.values
        dk['lat'].attrs["units"]='degrees_north'
        dk['lon'].attrs["units"]='degrees_east'
        dk['ar'].attrs["units"]='AR binary array'
        
        try:
            os.remove(dir2+'era5.ar.reid500.'+str(year)+'.nc')
        except OSError:
            pass
        dk.to_netcdf(dir2+'era5.ar.reid500.'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir2+'merra2.ar.labels.model.1980.nc')
labels=ds['ar_labeled']

yrs=np.arange(1980,2020,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year) 
    
    year_length=365
    if year%4==0:
        year_length=366
        
    dates=[dt.datetime(year,1,1,0,30)+dt.timedelta(hours=6*x) for x in range(4*year_length)]

    if 1==1:
        #ds=xr.open_dataset('/Users/ahenny/Downloads/MERRA2.ar_tag.Lora_v2.1hr.'+str(year)+'0101-'+str(year)+'1231.nc')
        #ds=xr.open_dataset('/Users/ahenny/Downloads/MERRA2_native.ar_tag.ARCONNECT_v2.1hr.'+str(year)+'0101-'+str(year)+'1231.nc')
        #ds=xr.open_dataset('/Users/ahenny/Downloads/MERRA2_native.ar_tag.ClimateNet_DL_model.1hr.'+str(year)+'0101-'+str(year)+'1231.nc')
        
        ar_concat=ds['ar_binary_tag'].sel(time=dates)
        ar_concat['lon']=labels.lon.values
        ar_concat['lat']=labels.lat.values
        
        dk=xr.Dataset()
        dk['ar']=(('time','lat','lon'),ar_concat.values)
        
        dk.coords['time']=ar_concat.time.values
        dk.coords['lat']=ar_concat.lat.values
        dk.coords['lon']=ar_concat.lon.values
        dk['lat'].attrs["units"]='degrees_north'
        dk['lon'].attrs["units"]='degrees_east'
        dk['ar'].attrs["units"]='AR binary array'
        
        try:
            os.remove(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
        except OSError:
            pass
        dk.to_netcdf(dir2+'merra2.ar.cndl.'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
yrs=np.arange(1980,2020,1)
ds=xr.open_dataset('/Users/ahenny/Downloads/JRA55.ar_tag.GuanWaliser_v2.6hr.19800101-20191231.nc')
for i in range(len(yrs)):
    year=yrs[i]
    print(year) 
    
    year_length=365
    if year%4==0:
        year_length=366
        
    dates=[dt.datetime(year,1,1,0)+dt.timedelta(hours=6*x) for x in range(4*year_length)]

    if 1==1:
        ar_concat=ds['ar_binary_tag'].sel(time=dates)
        
        dk=xr.Dataset()
        dk['ar']=(('time','lat','lon'),ar_concat.values)
        dk.coords['time']=ar_concat.time.values
        dk.coords['lat']=ar_concat.lat.values
        dk.coords['lon']=ar_concat.lon.values
        dk['lat'].attrs["units"]='degrees_north'
        dk['lon'].attrs["units"]='degrees_east'
        dk['ar'].attrs["units"]='AR binary array'
        
        try:
            os.remove(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
        except OSError:
            pass
        dk.to_netcdf(dir2+'jra55.ar.gwv2.'+str(year)+'.nc',mode='w',format='NETCDF4')